In [1]:
from comet_ml import API

import pickle
import pandas as pd


In [2]:
#Download Model
api = API()
experiment = api.get("meriembchaaben/ift6758/6cc6d08bce9844af9141e87129e9a78f")
api.download_registry_model("meriembchaaben", "final-best-task-6-xgb", "1.0.0",
                            output_path="./", expand=True)

COMET INFO: Downloading registry model 'final-best-task-6-xgb', version '1.0.0', stage None from workspace 'meriembchaaben'...
COMET INFO: Unzipping model to '/home/smuchi/Dev/IFT6758/notebooks' ...
COMET INFO: done!


In [3]:
# Load Model
with open(f'./sklearn-XGB-CV-NFS.pkl', 'rb') as f:
    pipe = pickle.load(f)

In [4]:
# Load Test Data
PATH_REL_PREFIX = '..'
df = pd.read_csv(f'{PATH_REL_PREFIX}/data/M2Data/testSet.csv')

# The following pre-processing code is not required to run the SKLearn Pipeline
# I kept it for reference
 
# df['strength'] = df['strength2']

# df['rebound'] = df.rebound.astype(int)
# df['Goal'] = df['Goal'].astype(int)
# df['EmptyNet'] = df['EmptyNet'].astype(int)

# df = df.drop(labels=[
#     'totalPlayTime', 'gameID', 'eventType',
#     'lastEventPeriodTime',
#     'strength2',
#     'lastEventAngle',
#     'lastEventGameSeconds',
#     'emptyNet',
#     'lastEventPeriod',
#     'xCoord', 'yCoord',
#     'lastEventXCoord', 'lastEventYCoord',
#     'index',
# ], axis=1, errors='ignore')

# X, y = df.drop(labels='Goal', axis=1), df['Goal']

In [5]:
# Make Predictions
test_actual_labels = df.Goal.to_numpy()
test_predicted_labels = pipe.predict(df)
test_predicted_probabilities = pipe.predict_proba(df)[:,1]

results_df = pd.DataFrame({
    'actual': test_actual_labels,
    'predicted_labels': test_predicted_labels,
    'predicted_probabilities': test_predicted_probabilities,
})

# Save Predictions to CSV
results_df.to_csv('./predictions/task-6-best-final.csv')

In [6]:
# Quick Validation of Test Set Predicitons
from sklearn.metrics import roc_auc_score, f1_score, brier_score_loss
print(roc_auc_score(test_actual_labels, test_predicted_probabilities))
print(f1_score(test_actual_labels, test_predicted_labels, average='macro'))
print(brier_score_loss(test_actual_labels, test_predicted_probabilities))

0.7912830588552948
0.6309400776047751
0.12252128006127017
